<a href="https://colab.research.google.com/github/Rohit9264/svm_pathway_normal_or_forwarding/blob/main/svm_randomForest_GridSearchCv_naive%20Bayes_random%20forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install imbalanced-learn


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

In [36]:
data=pd.read_csv("/content/datasetSMOTE.csv")
print(data)

        Serial_Number  Event        Time  S_Node  Node_id  Rest_Energy  \
0                   0      1    0.100000      79       79   600.000000   
1                   1      2    0.100963      78       78   599.979723   
2                   2      2    0.100963      76       76   599.979722   
3                   3      2    0.100964      75       75   599.979722   
4                   4      2    0.100964     118      118   599.979722   
...               ...    ...         ...     ...      ...          ...   
270491         312064      2  341.798498     114      114   529.152517   
270492         312065      2  341.798498      38       38   529.784757   
270493         312066      2  341.798499     113      113   529.079985   
270494         312067      2  341.798499      71       71   528.709811   
270495         312068      2  341.798499     119      119   529.468660   

        Trace_Level  Mac_Type_Pckt  Source_IP_Port  Des_IP_Port  Packet_Size  \
0                 5            

In [ ]:
print(data.columns)



Index(['Serial_Number', 'Event', 'Time', 'S_Node', 'Node_id', 'Rest_Energy',
       'Trace_Level', 'Mac_Type_Pckt', 'Source_IP_Port', 'Des_IP_Port',
       'Packet_Size', 'TTL', 'Hop_Count', 'Broadcast_ID', 'Dest_Node_Num',
       'Dest_Seq_Num', 'Src_Node_ID', 'Src_Seq_Num', 'Class'],
      dtype='object')


In [ ]:
data.describe()


,Serial_Number,Event,Time,S_Node,Node_id,Rest_Energy,Trace_Level,Mac_Type_Pckt,Source_IP_Port,Des_IP_Port,Packet_Size,TTL,Hop_Count,Broadcast_ID,Dest_Node_Num,Dest_Seq_Num,Src_Node_ID,Src_Seq_Num
count,270496.000000,270496.000000,270496.000000,270496.000000,270496.000000,270496.000000,270496.0,270496.000000,270496.000000,2.704960e+05,270496.000000,270496.000000,270496.000000,270496.000000,270496.000000,270496.000000,270496.000000,270496.000000
mean,166492.500913,2.234469,182.100172,99.866671,99.866671,562.103155,5.0,799.875784,99.997425,1.255000e+00,58.074781,22.629547,6.854948,31.677707,153.876320,1.178587,146.894220,66.925256
std,88673.270018,0.765386,95.069897,55.492469,55.492469,19.810710,0.0,9.967845,54.343081,3.536955e-12,21.973572,5.913511,2.909738,21.755071,27.170769,3.419895,40.149083,44.043364
min,0.000000,1.000000,0.100000,0.000000,0.000000,524.691852,5.0,0.000000,0.255000,1.255000e+00,48.000000,0.000000,1.000000,1.000000,36.000000,0.000000,0.000000,4.000000
25%,101206.750000,2.000000,102.547119,52.000000,52.000000,543.640004,5.0,800.000000,54.255000,1.255000e+00,48.000000,21.000000,5.000000,10.000000,137.000000,0.000000,160.000000,22.000000
50%,171378.500000,2.000000,175.661450,101.000000,101.000000,563.119858,5.0,800.000000,100.255000,1.255000e+00,48.000000,24.000000,7.000000,31.000000,152.000000,0.000000,160.000000,66.000000
75%,241561.250000,2.000000,272.047746,146.000000,146.000000,578.610421,5.0,800.000000,144.255000,1.255000e+00,48.000000,26.000000,9.000000,51.000000,175.000000,0.000000,160.000000,106.000000
max,312068.000000,4.000000,341.798499,199.000000,199.000000,600.000000,5.0,800.000000,199.255000,1.255000e+00,106.000000,30.000000,17.000000,70.000000,199.000000,17.000000,188.000000,144.000000


In [ ]:
data.shape


(270496, 19)

In [ ]:
data.head()



,Serial_Number,Event,Time,S_Node,Node_id,Rest_Energy,Trace_Level,Mac_Type_Pckt,Source_IP_Port,Des_IP_Port,Packet_Size,TTL,Hop_Count,Broadcast_ID,Dest_Node_Num,Dest_Seq_Num,Src_Node_ID,Src_Seq_Num,Class
0,0,1,0.100000,79,79,600.000000,5,0,79.255,1.255,48,30,1,1,100,0,79,4,normal
1,1,2,0.100963,78,78,599.979723,5,800,79.255,1.255,48,30,1,1,100,0,79,4,normal
2,2,2,0.100963,76,76,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,normal
3,3,2,0.100964,75,75,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,normal
4,4,2,0.100964,118,118,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,normal


In [ ]:
data['Class'].value_counts()

,count
Class,
normal,262851
Forwarding,7645


In [ ]:

#label 0 =>malicious
#label 1 =>normal
data.groupby('Class').mean()

,Serial_Number,Event,Time,S_Node,Node_id,Rest_Energy,Trace_Level,Mac_Type_Pckt,Source_IP_Port,Des_IP_Port,Packet_Size,TTL,Hop_Count,Broadcast_ID,Dest_Node_Num,Dest_Seq_Num,Src_Node_ID,Src_Seq_Num
Class,,,,,,,,,,,,,,,,,,
Forwarding,153313.576586,2.232963,169.452610,73.040157,73.040157,562.74695,5.0,799.790713,74.454215,1.255,55.776324,24.205755,4.181033,27.462917,141.770700,1.617659,127.782603,58.365729
normal,166875.808857,2.234513,182.468026,100.646918,100.646918,562.08443,5.0,799.878258,100.740347,1.255,58.141632,22.583703,6.932719,31.800294,154.228411,1.165816,147.450080,67.174209


In [37]:
#separaring data and labels
# drop_cols = [
#     'Event',
#     'Node_id',
#     'Trace_Level',
#     'Mac_Type_Pckt',
#     'Source_IP_Port',
#     'Des_IP_Port',
#     'Broadcast_ID',
#     'Dest_Seq_Num',
#     'Src_Node_ID',
#     'Src_Seq_Num'
# ]
x = data.drop(columns=['Class', 'Serial_Number'],axis=1 )
y=data['Class']
print(x)
print(y)
x.shape

        Event        Time  S_Node  Node_id  Rest_Energy  Trace_Level  \
0           1    0.100000      79       79   600.000000            5   
1           2    0.100963      78       78   599.979723            5   
2           2    0.100963      76       76   599.979722            5   
3           2    0.100964      75       75   599.979722            5   
4           2    0.100964     118      118   599.979722            5   
...       ...         ...     ...      ...          ...          ...   
270491      2  341.798498     114      114   529.152517            5   
270492      2  341.798498      38       38   529.784757            5   
270493      2  341.798499     113      113   529.079985            5   
270494      2  341.798499      71       71   528.709811            5   
270495      2  341.798499     119      119   529.468660            5   

        Mac_Type_Pckt  Source_IP_Port  Des_IP_Port  Packet_Size  TTL  \
0                   0          79.255        1.255           48

(270496, 17)

In [ ]:
# Step 1: Fix typos in class labels
y = y.replace('nor', 'normal')  # replace wrong label with correct one

# Step 2: Remove any class that has fewer than 2 samples
class_counts = y.value_counts()
valid_classes = class_counts[class_counts >= 2].index
mask_valid_classes = y.isin(valid_classes)

x = x[mask_valid_classes]
y = y[mask_valid_classes]

In [ ]:

# 1. Encode labels to 0/1
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # normal -> 1, Forwarding -> 0 (or vice-versa)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)


In [ ]:
# 3. Balance training data with SMOTE
smote = SMOTE(random_state=42)
x_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)


In [ ]:
# Check the new distribution of classes after applying SMOTE
print(pd.Series(y_train_smote).value_counts())

1    210280
0    210280
Name: count, dtype: int64


In [ ]:
print('Before SMOTE:', x_train.shape, y_train.shape)
print('After SMOTE:', x_train_smote.shape, y_train_smote.shape)

Before SMOTE: (216396, 7) (216396,)
After SMOTE: (420560, 7) (420560,)


##svm

In [ ]:
# 4. Train a faster SVM
svm_model = LinearSVC(max_iter=10000)  # you can increase max_iter if needed
svm_model.fit(x_train_smote, y_train_smote)



LinearSVC(max_iter=10000)

In [ ]:
# 5. Predict
y_pred = svm_model.predict(x_test)

In [38]:
# 6. Evaluate
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:\n', classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.7946256467485036
Classification Report:
               precision    recall  f1-score   support

  Forwarding       0.96      0.72      0.82     52571
      normal       0.63      0.94      0.75     26285

    accuracy                           0.79     78856
   macro avg       0.79      0.83      0.79     78856
weighted avg       0.85      0.79      0.80     78856



Prediction: [0]
Normal node ✅


##test data



In [40]:
new_input_data = (2, 0.101383814, 43, 43, 599.979638, 5, 800, 0.255, 1.255, 48, 30, 1, 1, 121, 0, 0, 4,0.0)

# Convert the input to a NumPy array and reshape for prediction
input_as_numpy_array = np.asarray(new_input_data).reshape(1, -1)

# Make the prediction using the trained classifier
prediction = svm_model.predict(input_as_numpy_array)

# Output the prediction
print("Prediction:", prediction)

# Interpret the prediction (0 = normal, 1 = malicious)
if prediction == 0:
    print("Normal node")
else:
    print("Malicious node")

Prediction: [0]
Normal node


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


In [41]:
input_data = (2, 0.103838522, 43, 43, 599.979068, 5, 800, 3.255, 1.255, 48, 29, 2, 1, 121, 0, 0, 4)

# Convert the input to a NumPy array and reshape for prediction
input_as_numpy_array = np.asarray(new_input_data).reshape(1, -1)

# Make the prediction using the trained classifier
prediction = svm_model.predict(input_as_numpy_array)

# Output the prediction
print("Prediction:", prediction)

# Interpret the prediction (0 = normal, 1 = malicious)
if prediction == 0:
    print("Normal node")
else:
    print("Malicious node")

Prediction: [0]
Normal node


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


##random forest


In [42]:
# Retrain the classifier on the balanced dataset
classifier = RandomForestClassifier(n_estimators=500, max_depth=100, random_state=42)
classifier.fit(x_train_smote, y_train_smote)

RandomForestClassifier(max_depth=100, n_estimators=500, random_state=42)

In [43]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict on the test set (or training set if using cross-validation)
y_pred = classifier.predict(x_test)  # Replace x_test with the appropriate test set

# Print classification report for detailed metrics
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=['Normal', 'Forwarding']))

# Print confusion matrix to see how well the model is classifying each class
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

      Normal       1.00      0.99      1.00     52571
  Forwarding       0.99      1.00      0.99     26285

    accuracy                           1.00     78856
   macro avg       0.99      1.00      0.99     78856
weighted avg       1.00      1.00      1.00     78856

Confusion Matrix:
 [[52297   274]
 [   86 26199]]


In [44]:
misclassified = x_test[(y_pred != y_test)]
print(misclassified)

        Unnamed: 0  Event        Time  S_Node  Node_id  Rest_Energy  \
214792      253937      2  279.621567      81       81   538.527308   
366809      165613      2  170.629163      45       45   562.803301   
174563      212200      2  244.626810      81       81   546.215586   
139556      175872      2  185.612655      81       81   559.154840   
384339      274123      2  295.626129      48       48   535.296481   
...            ...    ...         ...     ...      ...          ...   
305010       35711      2   70.617262      45       45   584.706723   
197689      236224      2  270.107977      81       81   540.612545   
296524      148508      2  150.637646      49       49   567.153159   
102638      137537      2  140.634141      81       81   569.039778   
57              59      2    0.103814      46       46   599.979068   

        Trace_Level  Mac_Type_Pckt  Source_IP_Port  Des_IP_Port  Packet_Size  \
214792            5            800      122.255000        1.255    

In [46]:
input_data = (2, 0.103838522, 43, 43, 599.979068, 5, 800, 3.255, 1.255, 48, 29, 2, 1, 121, 0, 0, 4,0.0)

# Convert the input data to a NumPy array and reshape for prediction
input_as_numpy_array = np.asarray(input_data).reshape(1, -1)

# Make the prediction using the trained classifier
prediction = classifier.predict(input_as_numpy_array)

# Output the prediction result
print("Predicted class:", prediction)

# Interpret the prediction (you can map the output back to your class labels, e.g., 0 for 'Normal' and 1 for 'Forwarding')
if prediction == 0:
    print("Normal node")
elif prediction == 1:
    print("Forwarding node")  # Adjust this based on your label encoding

Predicted class: [0]
Normal node


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


##grid search cv



In [47]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=3)

# Fit the grid search
grid_search.fit(x_train_smote, y_train_smote)

# Best parameters from GridSearchCV
print(f"Best Parameters: {grid_search.best_params_}")

# Train the classifier with the best parameters
best_classifier = grid_search.best_estimator_
best_classifier.fit(x_train_smote, y_train_smote)


KeyboardInterrupt: 

Naive Bayes

In [51]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Load the dataset
df = pd.read_csv('/content/datasetSMOTE.csv')

# Preview the dataset structure
print("Dataset Head:\n", df.head())

# Separate features (X) and label (y)
# Assuming the last column is the label
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Encode labels (e.g., "Normal", "Forwarding", etc.)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Initialize and train the Naive Bayes classifier
model = GaussianNB()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluation
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# --------------------------
# Predict a single new sample
# --------------------------
# Example input (must match the number of features)
new_input_data = (2, 244.6219962, 84, 84, 546.176863, 5, 800, 127.255, 1.255, 48, 27, 4, 48, 186, 0, 160, 100,0.0)
input_np = np.asarray(new_input_data).reshape(1, -1)

prediction = model.predict(input_np)
predicted_label = label_encoder.inverse_transform(prediction)

print("Prediction for new input:", predicted_label[0])


Dataset Head:
    Serial_Number  Event      Time  S_Node  Node_id  Rest_Energy  Trace_Level  \
0              0      1  0.100000      79       79   600.000000            5   
1              1      2  0.100963      78       78   599.979723            5   
2              2      2  0.100963      76       76   599.979722            5   
3              3      2  0.100964      75       75   599.979722            5   
4              4      2  0.100964     118      118   599.979722            5   

   Mac_Type_Pckt  Source_IP_Port  Des_IP_Port  Packet_Size  TTL  Hop_Count  \
0              0          79.255        1.255           48   30          1   
1            800          79.255        1.255           48   30          1   
2            800          79.255        1.255           48   30          1   
3            800          79.255        1.255           48   30          1   
4            800          79.255        1.255           48   30          1   

   Broadcast_ID  Dest_Node_Num  Des

Logistic Regression

In [53]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Load the dataset
df = pd.read_csv('/content/datasetSMOTE.csv')

# Preview the dataset
print("Dataset Head:\n", df.head())

# Split features and label
# Assuming the last column is the label
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Encode string labels to numeric
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Initialize and train the Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluation
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# --------------------------
# Predicting a new sample
# --------------------------
# Example input — make sure this has the same number of features as your dataset
new_input_data = (2, 244.6219962, 84, 84, 546.176863, 5, 800, 127.255, 1.255, 48, 27, 4, 48, 186, 0, 160, 100,0.0)
input_np = np.asarray(new_input_data).reshape(1, -1)

prediction = model.predict(input_np)
predicted_label = label_encoder.inverse_transform(prediction)

print("Prediction for new input:", predicted_label[0])


Dataset Head:
    Serial_Number  Event      Time  S_Node  Node_id  Rest_Energy  Trace_Level  \
0              0      1  0.100000      79       79   600.000000            5   
1              1      2  0.100963      78       78   599.979723            5   
2              2      2  0.100963      76       76   599.979722            5   
3              3      2  0.100964      75       75   599.979722            5   
4              4      2  0.100964     118      118   599.979722            5   

   Mac_Type_Pckt  Source_IP_Port  Des_IP_Port  Packet_Size  TTL  Hop_Count  \
0              0          79.255        1.255           48   30          1   
1            800          79.255        1.255           48   30          1   
2            800          79.255        1.255           48   30          1   
3            800          79.255        1.255           48   30          1   
4            800          79.255        1.255           48   30          1   

   Broadcast_ID  Dest_Node_Num  Des

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
